<a href="https://colab.research.google.com/github/david3080/llmapi/blob/main/2_ragwithllamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndexを使ったRAGパイプライン

1. LlamaIndex関連ライブラリのインストール
2. EmbeddingモデルをロードとRAG対象データのロードとインデックス化
3. QAプロンプトとRefineプロンプトのカスタマイズ
4. RAGを使った問い合わせを行う（4種のLLM活用）

### 1. LlamaIndex関連ライブラリのインストール

AuthropicのClaude3とOpenAIのGPT-4、GoogleのGemini、GroqのLlama3をLLMモデルとして利用し、Embedding用モデルはHuggingFaceのモデルを利用する。llama-indexは破壊的変更があったりするので、最新の[0.10](https://note.com/npaka/n/nb8acc1f63312)を指定しています。

In [ ]:
%pip install llama-index==0.10.40 llama-index-llms-anthropic llama-index-llms-openai llama-index-llms-gemini llama-index-llms-groq llama-index-embeddings-huggingface python-dotenv

### 2. EmbeddingモデルをロードとRAG対象データのロードとインデックス化

Embeddingには多言語E5(multilingual-e5)を利用します。RAG対象データとしては、[ポール・グレアムのエッセイの日本語訳](https://raw.githubusercontent.com/david3080/llmapi/main/data/paul_graham_essay.txt)をネットからダウンロードしてインデックス化します。

In [ ]:
!mkdir -p 'data/1_BasicRAG'
!wget 'https://raw.githubusercontent.com/david3080/llmapi/main/data/paul_graham_essay.txt' -O 'data/1_BasicRAG/paul_graham_essay.txt'

In [ ]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# 多言語E5モデルをロードしてRAG対象データをEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")

from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
)
Settings.embed_model = embed_model
Settings.chunk_size = 1024
documents = SimpleDirectoryReader("./data/1_BasicRAG").load_data()
index = VectorStoreIndex.from_documents(documents)

### 3. QAプロンプトとRefineプロンプトのカスタマイズ

※ 参考: https://docs.llamaindex.ai/en/stable/examples/customization/prompts/chat_prompts/

In [27]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "コンテキスト情報は下記のとおりです。\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "事前知識ではなく、常に提供されたコンテキスト情報を使用してクエリに回答してください。: {query_str}\n"
)

refine_prompt_str = (
    "以下の追加のコンテキストを使用して必要な場合のみ元の回答を改良します.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "新しいコンテキストを考慮して元の回答を改良して質問に対するより適切な回答をします: {query_str}. "
    "コンテキストが役に立たない場合は元の回答を再度出力してください.\n"
    "元の回答: {existing_answer}"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "コンテキストが役に立たない場合でも必ず質問に日本語で答えてください."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "コンテキストが役に立たない場合でも必ず質問に答えてください."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=refine_prompt_str),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

### 4. RAGを使った問い合わせを行う（4種のLLM活用）

groqのLlama3が一番レスポンスが早く、次にGPT-4、Gemini、Claude3の順番。

- OpenAI https://docs.llamaindex.ai/en/stable/examples/llm/openai/
- Anthropic https://docs.llamaindex.ai/en/stable/examples/llm/anthropic/
- Gemini https://docs.llamaindex.ai/en/stable/examples/llm/gemini/
- Groq https://docs.llamaindex.ai/en/stable/examples/llm/groq/

In [8]:
import os
from dotenv import load_dotenv
if load_dotenv() == False:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [9]:
query_str = "著者は大学で何を学びましたか？"

In [ ]:
# AnthropicのClaude3を使ったクエリ
from llama_index.llms.anthropic import Anthropic
Settings.llm = Anthropic(model="claude-3-opus-20240229")
query_engine = index.as_query_engine(text_qa_template=text_qa_template,refine_template=refine_template)
response = query_engine.query(query_str)
print(response)

In [ ]:
# OpenAIのGPT-4を使ったクエリ
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4")
query_engine = index.as_query_engine(text_qa_template=text_qa_template,refine_template=refine_template)
response = query_engine.query(query_str)
print(response)

In [ ]:
# GoogleのGeminiを使ったクエリ
from llama_index.llms.gemini import Gemini
Settings.llm = Gemini(model_name="models/gemini-pro")
query_engine = index.as_query_engine(text_qa_template=text_qa_template,refine_template=refine_template)
response = query_engine.query(query_str)
print(response)

In [ ]:
# GroqのLlama3を使ったクエリ
from llama_index.llms.groq import Groq
Settings.llm = Groq(model="llama3-70b-8192")
query_engine = index.as_query_engine(text_qa_template=text_qa_template,refine_template=refine_template)
response = query_engine.query(query_str)
print(response)